In [67]:
%load_ext autoreload
%autoreload 2

import loading

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
ignored_logs = [
    "loaded_scripts.log",
    "capture_loss.log",
    "stats.log",
    "packet_filter.log",
]
zeek_logs = loading.load_all_zeek_logs("../stratosphere-work-challenge-v1/zeek",ignored_logs) 
# change this to the path of the zeek logs

merged_df = loading.merge_logs(
    zeek_logs, primary_log="conn"
)  # primary log is the log that will be used as the base for the merge
print(merged_df.head())

Loading conn.log...
Loading dns.log...
Loading files.log...
Loading http.log...
Loading ssl.log...
Loading x509.log...
             ts                 uid   id.orig_h  id.orig_p id.resp_h  \
0  1.599057e+09  CMLZbu3FDJYoZwa27k  10.8.0.117       1210   8.8.8.8   
1  1.599057e+09  CgRCjV3z8dKmNVIvhb  10.8.0.117      43814   8.8.8.8   
2  1.599057e+09  CgFfWv3PUApAZUINNf  10.8.0.117      51631   8.8.8.8   
3  1.599057e+09  CIkFu02IznJPZcp1El  10.8.0.117      65449   8.8.8.8   
4  1.599057e+09  Cylq6E2mc9lVLjs8ua  10.8.0.117      63247   8.8.8.8   

   id.resp_p proto service  duration orig_bytes  ... next_protocol  \
0         53   udp     dns  0.010542         31  ...           NaN   
1         53   udp     dns  0.010908         33  ...           NaN   
2         53   udp     dns  0.010734         50  ...           NaN   
3         53   udp     dns  0.010405         37  ...           NaN   
4         53   udp     dns  0.009332         40  ...           NaN   

  established cert_chain_fu

In [75]:
print("flows in merged log file: ", len(merged_df))
print("columns: ", len(merged_df.columns), merged_df.columns)

# save for inspection
merged_df.to_csv("merged_zeek_logs.csv", index=False)

flows in merged log file:  234
columns:  96 Index(['ts', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto',
       'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state',
       'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts',
       'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents',
       'log_type', 'ts_dns', 'id.orig_h_dns', 'id.orig_p_dns', 'id.resp_h_dns',
       'id.resp_p_dns', 'proto_dns', 'trans_id', 'rtt', 'query', 'qclass',
       'qclass_name', 'qtype', 'qtype_name', 'rcode', 'rcode_name', 'AA', 'TC',
       'RD', 'RA', 'Z', 'answers', 'TTLs', 'rejected', 'log_type_dns',
       'ts_http', 'id.orig_h_http', 'id.orig_p_http', 'id.resp_h_http',
       'id.resp_p_http', 'trans_depth', 'method', 'host', 'uri', 'referrer',
       'version', 'user_agent', 'origin', 'request_body_len',
       'response_body_len', 'status_code', 'status_msg', 'info_code',
       'info_msg', 'tags', 'username', 'password', 'proxied', 'orig_fuids',


In [ ]:
merged_df.drop(columns=["uid"], inplace=True)
merged_df = convert_ip_addresses(merged_df, ["id.orig_h", "id.resp_h"])

In [ ]:
print("columns with only one unique value and drop them:")
for col in merged_df.columns:
    # print(col, len(merged_df[col].unique()))
    if len(merged_df[col].unique()) == 1:
        print(col)
        merged_df.drop(columns=[col], inplace=True)


columns with only one unique value:
local_orig
local_resp
missed_bytes
tunnel_parents
tags


In [ ]:
print("remove columns which jusrt announce the log file name/type (dns etc.)")
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains("log_type")]
print("columns after removal: ",len(merged_df.columns),"\n", merged_df.columns)

columns after removal:  88 
 Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts',
       'resp_ip_bytes', 'ts_dns', 'id.orig_h_dns', 'id.orig_p_dns',
       'id.resp_h_dns', 'id.resp_p_dns', 'proto_dns', 'trans_id', 'rtt',
       'query', 'qclass', 'qclass_name', 'qtype', 'qtype_name', 'rcode',
       'rcode_name', 'AA', 'TC', 'RD', 'RA', 'Z', 'answers', 'TTLs',
       'rejected', 'ts_http', 'id.orig_h_http', 'id.orig_p_http',
       'id.resp_h_http', 'id.resp_p_http', 'trans_depth', 'method', 'host',
       'uri', 'referrer', 'version', 'user_agent', 'origin',
       'request_body_len', 'response_body_len', 'status_code', 'status_msg',
       'info_code', 'info_msg', 'username', 'password', 'proxied',
       'orig_fuids', 'orig_filenames', 'orig_mime_types', 'resp_fuids',
       'resp_filenames', 'resp_mime_types', 'ts_ssl', '

In [ ]:
# Convert T/F columns to 1/0 and make them numerical
tf_columns = merged_df.select_dtypes(include=['object']).columns
for col in tf_columns:
    if merged_df[col].isin(['T', 'F']).all():
        merged_df[col] = merged_df[col].map({'T': 1, 'F': 0}).astype(int)


             ts                 uid   id.orig_h  id.orig_p id.resp_h  \
0  1.599057e+09  CMLZbu3FDJYoZwa27k  10.8.0.117       1210   8.8.8.8   
1  1.599057e+09  CgRCjV3z8dKmNVIvhb  10.8.0.117      43814   8.8.8.8   
2  1.599057e+09  CgFfWv3PUApAZUINNf  10.8.0.117      51631   8.8.8.8   
3  1.599057e+09  CIkFu02IznJPZcp1El  10.8.0.117      65449   8.8.8.8   
4  1.599057e+09  Cylq6E2mc9lVLjs8ua  10.8.0.117      63247   8.8.8.8   

   id.resp_p proto service  duration orig_bytes  ... last_alert next_protocol  \
0         53   udp     dns  0.010542         31  ...        NaN           NaN   
1         53   udp     dns  0.010908         33  ...        NaN           NaN   
2         53   udp     dns  0.010734         50  ...        NaN           NaN   
3         53   udp     dns  0.010405         37  ...        NaN           NaN   
4         53   udp     dns  0.009332         40  ...        NaN           NaN   

  established  cert_chain_fuids  client_cert_chain_fuids  subject  issuer  \
0  

In [ ]:
merged_df.describe(include="all")